### Horoscopo Clarín y La Nación*

*para desmentirlo

In [107]:
import requests as rq
from lxml import html
import re
from numpy.random import uniform
from time import sleep

In [51]:
signos = ['Aries', 'Tauro', 'Géminis',
         'Cáncer', 'Leo', 'Virgo',
         'Libra', 'Escorpio',
         'Sagitario','Capricornio',
         'Acuario', 'Piscis']

In [96]:
def get_clean_from_element(e):
    return re.sub('\s*\r*\n+', '\n', '\n'.join(e.itertext())).strip()

In [97]:
def get_clarin():
    url = "https://www.clarin.com/horoscopo"
    r = rq.get(url_clarin)
    dom = html.fromstring(r.content)
    signos = [e.text for e in dom.xpath('//h2[not(@*)]')]
    contents = [get_clean_from_element(e) for e in dom.xpath('//div[@class="item"]')]
    return {signos[i]:contents[i] for i in range(len(signos))}

In [111]:
def get_lanacion():
    horoscopo = {}
    for s in signos:
        url = f'https://www.lanacion.com.ar/horoscopo/{s}'
        r = rq.get(url)
        e = dom.xpath('//section[contains(@class, "hoy")]/article')[0]
        content = get_clean_from_element(e)
        horoscopo[s] = content
        sleep(uniform(0,2))
    return horoscopo

In [112]:
horoscopo_clarin = get_clarin()
horoscopo_lanacion = get_lanacion()

In [113]:
print(horoscopo_clarin[signos[0]])

Aries
del 21.03
al 20.04
Horóscopo de hoy:
 Tendrás dificultades para pensar de manera lógica y racional. Por lo que te sentirás molesto y manifestarás inseguridad.
Amor:
 Un miembro viejo de la familia está en problemas. Las fiestas de hoy te darán la oportunidad para llenar de ternura a esa persona.
Riqueza:
 No viene al caso guardar secretos sobre tu situación económica. Con un poco de imaginación, puedes sortear los problemas.
Bienestar:
 Alguien cercano tiene una personalidad tan fuerte que hay que cuidarse de no dejarse arrastrar por sus fantasías y proyectos.


In [114]:
print(horoscopo_lanacion[signos[0]])

Aproveche el día al máximo, ya que contará con los suficientes impulsos para resolver todos los asuntos que hace tiempo posterga. Tome coraje y comience hoy.
Amor:
Por más que se sienta agredido por los celos de su enamorado, escúchelo. Intente comprenderlo, ya que usted no ha tenido una buena actitud hacia él.
Riqueza:
Si bien muchos de los cambios que surgirán en el trabajo no lo afectarán a usted de forma directa, procure estar precavido. Piense bien antes de actuar.
Bienestar:
Procure organizar una escapada para el fin de semana en familia, ya que volverán renovados. Busque el destino y compare precios, evite endeudarse.


# Cuentos Borges y Cortazar 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import numpy as np

In [2]:
url_cortazar = 'http://ciudadseva.com/autor/julio-cortazar/cuentos/'
url_borges = 'http://ciudadseva.com/autor/jorge-luis-borges/cuentos/'

In [7]:
def get_cuentos(url):
    titulos = []
    textos = []
    
    response = requests.get(url)

    for s_url in re.findall('(http://ciudadseva.com/texto/.+?/)', response.text):   
        cuento = requests.get(s_url)
        soup = BeautifulSoup(cuento.content, 'html.parser')

        for elem in soup.findAll("div", { "class" : "text-justify" })[:2]:
            cuento = elem.text
            
        titulos.append(s_url.split('/')[-2])
        textos.append(cuento)
        time.sleep(np.random.randint(1,4))
        
    return {k:v for k,v in zip(titulos, textos)}

In [8]:
cuentos_bor = get_cuentos(url_borges)
cuentos_cor = get_cuentos(url_cortazar)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
countv_bor = CountVectorizer()
countv_cor = CountVectorizer()

countv_bor.fit(cuentos_bor)
countv_cor.fit(cuentos_cor)

df_bor = pd.DataFrame(countv_bor.transform(cuentos_bor).toarray(), columns= countv_bor.get_feature_names())
df_cor = pd.DataFrame(countv_cor.transform(cuentos_cor).toarray(), columns= countv_cor.get_feature_names())

In [11]:
df_bor.head()

,1829,1874,abel,adivino,adrogue,aleph,alguien,andres,argumentum,armoa,...,tertius,tlon,total,trama,tres,un,uqbar,versiones,yo,zunz
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df_cor.head()

,68,77,agua,almuerzo,almuerzos,amarilla,amor,arriba,asesina,autopista,...,teoria,testigos,tibia,tomada,un,una,vaso,velorios,veridica,vialidad
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## Bonus:
    Usar time para dar al scraper un tiempo de espera tomado de una distibución uniforme de rango 3.
    Usar CountVectorizer de sci-kit learn para analizar los cuentos.


## Otros

1) 'https://weworkremotely.com/categories/2-programming/jobs' 
    Este sitio enumera ofertas de trabajo de programación del último mes. Armá un diccionario (u otra estructura) con los títulos de los trabajos, y con más información recolectada de cada link.
    
2) 'https://www.ncbi.nlm.nih.gov/pubmed/?term=dopamine'
    Usá el buscador de PubMed para descargar abstracts de múltiples estudios (por ejemplo de la primer página... o más! It's up to you!). Maneja la información convenientemente, por ejemplo guardandola en uno o más archivos. 

3) 'https://github.com/jarikoi/interesting-papers' o 'https://people.eecs.berkeley.edu/~alig/papers/'
    En estas páginas hay muchos PDFs. Descarga todos los que puedas! (un tip: en github, mira los links!
    